# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

## Env setup

In [ ]:
# This is needed to display the images.
%matplotlib inline
!pip install tensorflow-gpu==1.15.0

## Object detection imports
Here are the imports from the object detection module.

In [ ]:
%cd /content/drive/My Drive/obj_detec/models/research
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/content/drive/My Drive/obj_detec/models/research


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [ ]:

CKPT = '/content/drive/MyDrive/obj_detec/models/research/Training_record/Output_K136N/frozen_inference_graph.pb'

PATH_TO_LABELS = '/content/drive/MyDrive/obj_detec/models/research/Training_record/tf_record/label_map.pbtxt'

NUM_CLASSES = 50

## Load a (frozen) Tensorflow model into memory.

In [ ]:
import tensorflow.compat.v1 as tf

detection_graph = tf.Graph()

with detection_graph.as_default():
    
  od_graph_def = tf.GraphDef()

  with tf.gfile.GFile(CKPT, 'rb') as fid:
        
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'class-1'}, 2: {'id': 2, 'name': 'class-2'}, 3: {'id': 3, 'name': 'class-3'}, 4: {'id': 4, 'name': 'class-4'}, 5: {'id': 5, 'name': 'class-5'}, 6: {'id': 6, 'name': 'class-6'}, 7: {'id': 7, 'name': 'class-7'}, 8: {'id': 8, 'name': 'class-8'}, 9: {'id': 9, 'name': 'class-9'}, 10: {'id': 10, 'name': 'class-10'}, 11: {'id': 11, 'name': 'class-11'}, 12: {'id': 12, 'name': 'class-12'}, 13: {'id': 13, 'name': 'class-13'}, 14: {'id': 14, 'name': 'class-14'}, 15: {'id': 15, 'name': 'class-15'}, 16: {'id': 16, 'name': 'class-16'}, 17: {'id': 17, 'name': 'class-17'}, 18: {'id': 18, 'name': 'class-18'}, 19: {'id': 19, 'name': 'class-19'}, 20: {'id': 20, 'name': 'class-20'}, 21: {'id': 21, 'name': 'class-21'}, 22: {'id': 22, 'name': 'class-22'}, 23: {'id': 23, 'name': 'class-23'}, 24: {'id': 24, 'name': 'class-24'}, 25: {'id': 25, 'name': 'class-25'}, 26: {'id': 26, 'name': 'class-26'}, 27: {'id': 27, 'name': 'class-27'}, 28: {'id': 28, 'name': 'class-28'}, 29: {'id': 29, 'na

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [ ]:
from glob import glob

In [ ]:

PATH_TO_TEST_IMAGES_DIR = '/content/drive/MyDrive/obj_detec/models/research/Dataset/car/test'

#print(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))
#TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))  # PNG OR JPG
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))
print("Length of test images:", len(TEST_IMAGE_PATHS))

print(TEST_IMAGE_PATHS[2][66:-4])
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

Length of test images: 3936
1479504033347862726


In [ ]:
import time
import json
counter = 0
with detection_graph.as_default():
    time0 = time.time()
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        for image_path in TEST_IMAGE_PATHS:
            counter = counter + 1
            print('Image {}'.format(counter))
            filename_string = image_path[66:-4]
            print(filename_string)
            try:
              image = Image.open(image_path)
              # print(image.size)
              # the array based representation of the image will be used later in order to prepare the
              # result image with boxes and labels on it.
              image_np = load_image_into_numpy_array(image)
              # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
              image_np_expanded = np.expand_dims(image_np, axis=0)

              #time0 = time.time()

              # Actual detection.
              (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

              

              # Visualization of the results of a detection.
              # vis_util.visualize_boxes_and_labels_on_image_array(
              #  image_np,
              #  np.squeeze(boxes),
              #  np.squeeze(classes).astype(np.int32),
              #  np.squeeze(scores),
              #  category_index,
              #  use_normalized_coordinates=True,
              #  line_thickness=8)
              #plt.figure(figsize=IMAGE_SIZE)
              #plt.imshow(image_np)
              #plt.show()
              
              coordinates = vis_util.return_coordinates(
                          image,
                          np.squeeze(boxes),
                          np.squeeze(classes).astype(np.int32),
                          np.squeeze(scores),
                          category_index,
                          use_normalized_coordinates=True,
                          line_thickness=8,
                          min_score_thresh=0.5)
              #textfile = open("json/"+filename_string+".json", "a")
              #textfile.write(json.dumps(coordinates))
              #textfile.write("\n")
              #print(coordinates)
              if coordinates == []:
                  with open('/content/drive/My Drive/obj_detec/models/research/Training_record/test/output_metric/' + filename_string + '.txt','a') as text_file:
                      text_file.close()
                  
              for coordinate in coordinates:
                  print(coordinate)
                  (xmax, xmin, ymax, ymin, accuracy, classification_num) = coordinate
                  label = classification_num
                  accuracy = accuracy/100
                  detection_box = (label, accuracy, xmin, ymin, xmax, ymax)
                  
                  #if label == 46:
                  #    light = 'Red'
                      
                  #if label == 47:
                  #    light = 'Yellow'
                      
                  #if label == 48:
                  #    light = 'Green'
                      
                  #if label == 49:
                  #    light = 'off'
                      
                  #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                  
                  #if label == 46 or label == 47 or label == 48 or label == 49: 
                  with open('/content/drive/MyDrive/obj_detec/models/research/Training_record/output_metric' + filename_string + '.txt','a') as text_file:
                      text_file.write(str(label) + ' ' + str(accuracy) + ' ' + str(xmin) + ' ' + str(ymin) + ' ' + str(xmax) + ' ' + str(ymax) + '\n')
                      text_file.close()    
                  
                  
                  
                  #print('{} {} {} {} {} {}'.format(label, accuracy, xmin, ymin, xmax, ymax))
                  print(detection_box)
                  #if str(label) != '46' and str(label) != '47' and str(label) != '48' and str(label) != '49':
                      
                          
              time1 = time.time()

              print("Time in milliseconds", (time1 - time0) * 1000) 
            except:
              print("######################################################")
              with open('/content/drive/MyDrive/obj_detec/models/research/Training_record/output_metric/corrupt_images_new.txt','a') as text_file:
                text_file.write(image_path + '\n')
                text_file.close()
                
             

Streaming output truncated to the last 5000 lines.
Image 804
1479505716961217368
[840, 1044, 573, 654, 99.99258518218994, 1]
(1, 0.9999258518218994, 1044, 654, 840, 573)
[887, 998, 491, 545, 99.98724460601807, 1]
(1, 0.9998724460601807, 998, 545, 887, 491)
[899, 1038, 716, 781, 99.98600482940674, 1]
(1, 0.9998600482940674, 1038, 781, 899, 716)
[888, 1081, 753, 856, 99.96838569641113, 1]
(1, 0.9996838569641113, 1081, 856, 888, 753)
[912, 980, 76, 143, 99.95706677436829, 1]
(1, 0.9995706677436829, 980, 143, 912, 76)
[907, 970, 260, 321, 99.95362758636475, 1]
(1, 0.9995362758636475, 970, 321, 907, 260)
[874, 969, 967, 1081, 99.93112087249756, 1]
(1, 0.9993112087249756, 969, 1081, 874, 967)
[877, 999, 681, 727, 99.91104006767273, 1]
(1, 0.9991104006767273, 999, 727, 877, 681)
[879, 945, 1054, 1191, 99.89230036735535, 1]
(1, 0.9989230036735535, 945, 1191, 879, 1054)
[897, 952, 448, 472, 99.87706542015076, 1]
(1, 0.9987706542015076, 952, 472, 897, 448)
[889, 984, 662, 689, 99.66415166854858,